#Imports



In [18]:
import tensorflow as tf
print(tf.__version__)
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from csv import writer
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv1D,MaxPooling1D,BatchNormalization
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import json
import matplotlib
from matplotlib import ticker as ticker
import matplotlib.pyplot as plt
import h5py
import os
from sklearn.metrics import f1_score

2.17.1


# Load data (Change filename - must end in .json)

In [19]:
filename="flipStress_p12_t0.5s.json"
data = json.load(open(filename))
trainX = np.array(data['X'],dtype=np.float32)
trainY=np.array(data['y'])

In [20]:
print(trainX.shape)
print(trainY.shape)

(200, 5000)
(200,)


In [21]:
trainX = np.expand_dims(trainX,axis=2)

In [22]:
input_data = trainX

minimum = np.amin(input_data)
maximum = np.amax(input_data)

input_data_norm = (input_data-minimum)/(maximum-minimum)

print(input_data_norm.shape)

input_data_norm = input_data_norm

(200, 5000, 1)


In [23]:
trainY= keras.utils.to_categorical(trainY)
print(trainY.shape)

(200, 2)


In [24]:
def model_create(x=None):
    # This depends on the number of Website
    numberOfWebsite = 2

    # This depends on the number of samples
    input = keras.Input(shape = (5000,1))

    x = layers.Conv1D(128, 3, activation="relu", padding="same")(input)
    x = layers.Conv1D(64, 3, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=3)(x)

    x = layers.Conv1D(32, 3, activation="relu", padding="same")(x)
    x = MaxPooling1D(pool_size=3)(x)

    x = layers.Dropout(0.3)(x)
    x = layers.Flatten()(x)

    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)

    x = Dense(numberOfWebsite,activation='softmax')(x)
    model = keras.Model(inputs = input, outputs = x)

    return model

In [25]:
model = model_create()
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_11 (InputLayer)          │ (None, 5000, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_33 (Conv1D)                   │ (None, 5000, 128)           │             512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_34 (Conv1D)                   │ (None, 5000, 64)            │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_22 (MaxPooling1D)      │ (None, 1666, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_35 (Conv1D)                   │ (None, 1666, 32)            │           6,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_23 (MaxPooling1D)      │ (None, 555, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_33 (Dropout)                 │ (None, 555, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_11 (Flatten)                 │ (None, 17760)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_33 (Dense)                     │ (None, 256)                 │       4,546,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_34 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_35 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,611,298 (17.59 MB)

 Trainable params: 4,611,298 (17.59 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=["accuracy"])

In [27]:
from sklearn.model_selection import StratifiedShuffleSplit
n_splits = 10
sss = StratifiedShuffleSplit(n_splits=n_splits, random_state=42)
all_accuracies = []
split_num = 1

for train_index, val_index in sss.split(input_data_norm, trainY.argmax(axis=1)):

    X_train_fold, X_val_fold = input_data_norm[train_index], input_data_norm[val_index]
    y_train_fold, y_val_fold = trainY[train_index], trainY[val_index]

    # Print the shapes of the training and validation sets
    print(f"Split {split_num}:")
    print(f"Training set shape: {X_train_fold.shape}")
    print(f"Validation set shape: {X_val_fold.shape}")

    # Create a new model for each split
    model = model_create()

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(
        X_train_fold, y_train_fold,
        epochs=100,
        validation_data=(X_val_fold, y_val_fold),
        verbose=0
    )

    # Evaluate the model on the validation data
    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)

    all_accuracies.append(val_accuracy)
    print(f"Split {split_num} accuracy: {val_accuracy:.4f}\n")


    split_num += 1

# Calculate the average accuracy and standard deviation over all splits
average_accuracy = np.mean(all_accuracies)
std_accuracy = np.std(all_accuracies)
print(f"Average accuracy over {n_splits} splits: {average_accuracy:.4f}")
print(f"Standard deviation over {n_splits} splits: {std_accuracy:.4f}")

Split 1:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 1 accuracy: 0.4500

Split 2:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 2 accuracy: 0.6000

Split 3:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 3 accuracy: 0.6000

Split 4:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 4 accuracy: 0.6000

Split 5:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 5 accuracy: 0.6000

Split 6:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 6 accuracy: 0.5500

Split 7:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 7 accuracy: 0.3000

Split 8:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 8 accuracy: 0.6500

Split 9:
Training set shape: (180, 5000, 1)
Validation set shape: (20, 5000, 1)
Split 9 accuracy: 0.5500

Split 10:
Training set shape: (180, 5000, 1)
V